# Pandas.plot() base training notebook

In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import gc

print(">>> Imports done")

>>> Imports done


## Loading and examining available Vega datasets 

In [12]:
# this is ChatGPT generated code; debugging and task clarification was mine =)

url = 'https://github.com/vega/vega-datasets/tree/main/data'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

csv_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.csv')]
json_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.json')]

datasets = []
err_datasets = []

print("="*20 + " CSV datasets in processing " + "="*20)
for link in tqdm(csv_links):
    url = 'https://raw.githubusercontent.com' + link.replace('/blob/', '/')
    try:
        df = pd.read_csv(url)
        datasets.append({
            'DF_Name': link.split('/')[-1],
            'DF_Number_of_Rows': df.shape[0],
            'DF_Number_of_Columns': df.shape[1],
            'DF': df
        })
    except:
        err_datasets.append(url)

print("\n" + "="*20 + " JSON datasets in processing " + "="*20)
for link in tqdm(json_links):
    url = 'https://raw.githubusercontent.com' + link.replace('/blob/', '/')
    try:
        df = pd.read_json(url)
        datasets.append({
            'DF_Name': link.split('/')[-1],
            'DF_Number_of_Rows': df.shape[0],
            'DF_Number_of_Columns': df.shape[1],
            'DF': df
        })
    except:
        err_datasets.append(url)

print("\n" +"="*20 + f" Datasets, that were not loaded: [{len(err_datasets)}] " +  "="*20)
for ds in err_datasets:
    print(ds)

==================== CSV datasets in processing ====================


100%|██████████| 22/22 [00:12<00:00,  1.82it/s]



==================== JSON datasets in processing ====================


100%|██████████| 45/45 [00:37<00:00,  1.20it/s]


==================== Datasets, that were not loaded: [7] ====================
https://raw.githubusercontent.com/vega/vega-datasets/main/data/annual-precip.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/earthquakes.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/londonBoroughs.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/londonTubeLines.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/miserables.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/us-10m.json
https://raw.githubusercontent.com/vega/vega-datasets/main/data/world-110m.json


A new issue has been opened for the Vega Team regarding datasets that cannot be loaded:
https://github.com/vega/vega-datasets/issues/438

In [13]:
df_datasets = pd.DataFrame(datasets).set_index('DF_Name')

del datasets
gc.collect()

0

In [37]:
df_datasets.to_pickle('data/vega_datasets.pkl')
# df_datasets = pd.read_pickle('data/vega_datasets.pkl')

In [17]:
df_datasets = df_datasets.drop('DF', axis=1) if 'DF' in df_datasets.columns else df_datasets

df_datasets.sort_values('DF_Number_of_Rows', ascending=False)[:10]

,DF_Number_of_Rows,DF_Number_of_Columns
DF_Name,,
flights-3m.csv,231083,5
flights-200k.json,231083,3
zipcodes.csv,42049,6
flights-20k.json,20000,5
flights-10k.json,10000,5
birdstrikes.csv,10000,14
seattle-weather-hourly-normals.csv,8759,4
jobs.json,7650,5
platformer-terrain.json,7514,8


In [13]:
err_datasets = [
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/annual-precip.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/earthquakes.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/londonBoroughs.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/londonTubeLines.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/miserables.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/us-10m.json',
'https://raw.githubusercontent.com/vega/vega-datasets/main/data/world-110m.json',
]

In [19]:
errs = []
for ds in err_datasets:
    try:
        pd.read_json(ds)
    except Exception as e:
        errs.append([type(e).__name__, str(e)])

errs

[['ValueError', 'All arrays must be of the same length'],
 ['ValueError', 'All arrays must be of the same length'],
 ['ValueError', 'All arrays must be of the same length'],
 ['ValueError', 'All arrays must be of the same length'],
 ['ValueError', 'All arrays must be of the same length'],
 ['ValueError',
  'Mixing dicts with non-Series may lead to ambiguous ordering.'],
 ['ValueError',
  'Mixing dicts with non-Series may lead to ambiguous ordering.']]